In [1]:
import sys, importlib
sys.path.append(r"C:\Users\DARY\Desktop\Dary\Prueba-BigData\Proyecto-hive\scripts")

import parametrossh
importlib.reload(parametrossh)

from parametrossh import get_params, init_spark, get_sqlserver_connection

In [4]:
# 1. Iniciar sesión de Spark
spark = init_spark(1, "CargaPedidos")
print("✅ SparkSession inicializada")


✅ SparkSession inicializada


In [5]:
# 2. Conexión SQL Server
jdbc_url, properties = get_sqlserver_connection(1)

# 3. Leer tabla Pedidos
df_pedidos = spark.read.jdbc(
    url=jdbc_url,
    table="dbo.Pedidos",
    properties=properties
)
print("✅ Pedidos desde SQL Server")
df_pedidos.show()

✅ Pedidos desde SQL Server


[Stage 0:>                                                          (0 + 1) / 1]

+--------+---------+------+-----------+--------------------+----------+
|PedidoID|ClienteID| Monto|FechaPedido|          CreateTime|UpdateTime|
+--------+---------+------+-----------+--------------------+----------+
|       1|        1|150.50| 2025-09-01|2025-09-14 19:36:...|      null|
|       2|        2|200.00| 2025-09-02|2025-09-14 19:36:...|      null|
|       3|        3| 50.00| 2025-09-03|2025-09-14 19:36:...|      null|
|       4|        4|300.00| 2025-09-04|2025-09-14 19:36:...|      null|
|       5|        5|120.00| 2025-09-05|2025-09-14 19:36:...|      null|
|       6|        1| 80.00| 2025-09-06|2025-09-14 19:36:...|      null|
|       7|        2| 60.00| 2025-09-07|2025-09-14 19:36:...|      null|
|       8|        3| 90.00| 2025-09-08|2025-09-14 19:36:...|      null|
|       9|        4|110.00| 2025-09-09|2025-09-14 19:36:...|      null|
|      10|        5| 75.00| 2025-09-10|2025-09-14 19:36:...|      null|
|      11|        7| 95.00| 2025-09-11|2025-09-14 19:36:...|    

In [6]:
df_pedidos.write.mode("overwrite").parquet("hdfs://namenode:8020/bronze/df_pedidos/")

In [7]:
df_validacion = spark.read.parquet("hdfs://namenode:8020/bronze/df_pedidos")
df_validacion.show(30)



[Stage 3:>                                                          (0 + 1) / 1]

+--------+---------+-------+-----------+--------------------+----------+
|PedidoID|ClienteID|  Monto|FechaPedido|          CreateTime|UpdateTime|
+--------+---------+-------+-----------+--------------------+----------+
|       1|        1| 150.50| 2025-09-01|2025-09-14 19:36:...|      null|
|       2|        2| 200.00| 2025-09-02|2025-09-14 19:36:...|      null|
|       3|        3|  50.00| 2025-09-03|2025-09-14 19:36:...|      null|
|       4|        4| 300.00| 2025-09-04|2025-09-14 19:36:...|      null|
|       5|        5| 120.00| 2025-09-05|2025-09-14 19:36:...|      null|
|       6|        1|  80.00| 2025-09-06|2025-09-14 19:36:...|      null|
|       7|        2|  60.00| 2025-09-07|2025-09-14 19:36:...|      null|
|       8|        3|  90.00| 2025-09-08|2025-09-14 19:36:...|      null|
|       9|        4| 110.00| 2025-09-09|2025-09-14 19:36:...|      null|
|      10|        5|  75.00| 2025-09-10|2025-09-14 19:36:...|      null|
|      11|        7|  95.00| 2025-09-11|2025-09-14 

In [3]:
df_validacion.describe()

DataFrame[summary: string, PedidoID: string, ClienteID: string, Monto: string]

In [4]:
df_validacion.printSchema()


root
 |-- PedidoID: integer (nullable = true)
 |-- ClienteID: integer (nullable = true)
 |-- Monto: decimal(10,2) (nullable = true)
 |-- FechaPedido: date (nullable = true)
 |-- CreateTime: timestamp (nullable = true)
 |-- UpdateTime: timestamp (nullable = true)



In [5]:
df = spark.read.parquet("hdfs://namenode:8020/bronze/df_pedidos")

# Forzamos tipos más compatibles con Hive
df_cast = (df
    .withColumn("Monto", df["Monto"].cast("double"))
    .withColumn("FechaPedido", df["FechaPedido"].cast("string"))  # Hive a veces falla con DATE en Parquet
)

df_cast.write.mode("overwrite").parquet("hdfs://namenode:8020/bronze/df_pedidos_hive")


In [8]:
spark.stop()
